<h3>Amendments Log</h3>
<table style="width:100%">
  <thead>
    <tr>
      <th style="text-align:left">Version</th>
      <th style="text-align:left">Amended By</th>
      <th style="text-align:left">Date</th>
      <th style="text-align:left">Description</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>1.2</td>
      <td>Gary Manley</td>
      <td>2025-12-02</td>
      <td>Moved posters to separate tabs with hyperlinks to improve readability.</td>
    </tr>
    <tr>
      <td>1.1</td>
      <td>Gary Manley</td>
      <td>2025-12-02</td>
      <td>Updated SQL to join Fact and Date Dim (Fixed missing release_date error).</td>
    </tr>
    <tr>
      <td>1.0</td>
      <td>Gary Manley</td>
      <td>2025-12-02</td>
      <td>Initial Version: Weekly Release Report with embedded posters.</td>
    </tr>
  </tbody>
</table>

In [ ]:
# 1. SETUP & IMPORTS
import duckdb
import pandas as pd
import xlsxwriter
import requests
import io
import os
import re
from datetime import datetime
from dotenv import load_dotenv

# Load Env
vLocalEnvPath = r"C:/Users/garym/Documents/GitHub/MovieReleases/.env"
if os.path.exists(vLocalEnvPath):
    load_dotenv(dotenv_path=vLocalEnvPath)
else:
    load_dotenv()

vMdToken = os.getenv("MOTHERDUCK_TOKEN")
if not vMdToken: raise RuntimeError("MOTHERDUCK_TOKEN missing")

# Connect
print("Connecting to MotherDuck...")
vCon = duckdb.connect(f"md:?motherduck_token={vMdToken}")

In [ ]:
# PARAMETERS / CONSTANTS
cNotebookName = "create_excel_report.ipynb"
vReportFilename = "Upcoming_Releases_Report.xlsx"

## 2. Fetch Data
Get the next 20 movies by joining Fact -> Film Dim -> Date Dim.

In [ ]:
print("Fetching upcoming releases...")
vSql = """
    SELECT 
        d.movie_title, 
        dt.date_actual as release_date, 
        d.poster_url,
        d.genres,
        d.cast_members
    FROM MovieReleases.silver.film_release_fact f
    JOIN MovieReleases.silver.film_release_dim d ON f.sk_film_release = d.sk_film_release
    JOIN MovieReleases.silver.date_dim dt ON f.sk_date = dt.sk_date
    WHERE dt.date_actual >= CURRENT_DATE()
    ORDER BY dt.date_actual ASC
    LIMIT 20
"""
dfReportData = vCon.sql(vSql).df()
print(f"Found {len(dfReportData)} movies.")

## 3. Generate Excel Report (Hyperlinked Tabs)
Instead of embedding images in the main list, we create a separate tab for each movie poster and link to it.

In [ ]:
print(f"Generating {vReportFilename}...")

vWorkbook = xlsxwriter.Workbook(vReportFilename)
vMainSheet = vWorkbook.add_worksheet("Summary List")

# --- FORMATS ---
fmt_header = vWorkbook.add_format({
    'bold': True, 'font_color': 'white', 'bg_color': '#2C3E50',
    'border': 1, 'align': 'center', 'valign': 'vcenter'
})
fmt_text = vWorkbook.add_format({'text_wrap': True, 'valign': 'top', 'border': 1})
fmt_date = vWorkbook.add_format({'num_format': 'dd mmm yyyy', 'valign': 'top', 'align': 'center', 'border': 1})
fmt_link = vWorkbook.add_format({'font_color': 'blue', 'underline': True, 'valign': 'top', 'border': 1, 'align': 'center'})
fmt_back_link = vWorkbook.add_format({'font_color': 'blue', 'underline': True, 'bold': True, 'font_size': 14})

# --- MAIN HEADERS ---
vHeaders = ["Release Date", "Movie Title", "Poster Link", "Genre", "Cast"]
for col_num, header in enumerate(vHeaders):
    vMainSheet.write(0, col_num, header, fmt_header)

vMainSheet.set_column('A:A', 15) # Date
vMainSheet.set_column('B:B', 30) # Title
vMainSheet.set_column('C:C', 15) # Link
vMainSheet.set_column('D:D', 20) # Genre
vMainSheet.set_column('E:E', 30) # Cast

# Helper to sanitize Excel sheet names (Max 31 chars, no illegal chars)
def f_sanitize_sheet_name(vTitle, vIndex):
    # Remove everything except Alphanumeric, Space, Underscore, Hyphen
    # This is much safer for JSON formatting than trying to escape complex symbols
    vClean = re.sub(r'[^A-Za-z0-9 _-]', '', str(vTitle))
    # Truncate to leave room for index (ensure uniqueness)
    vClean = vClean[:25]
    return f"{vIndex}_{vClean}"

# --- DATA ROWS ---
vRow = 1

for index, row in dfReportData.iterrows():
    # 1. Create Individual Movie Sheet
    vSheetName = f_sanitize_sheet_name(row['movie_title'], index + 1)
    vMovieSheet = vWorkbook.add_worksheet(vSheetName)
    
    # Add "Back to Summary" link at top
    vMovieSheet.write_url('A1', "internal:'Summary List'!A1", fmt_back_link, string="<< Back to Summary List")
    vMovieSheet.write('A3', row['movie_title'], fmt_back_link) # Title header
    
    # Insert Large Image
    vImgUrl = row['poster_url']
    vHasImage = False
    if vImgUrl and str(vImgUrl).startswith('http'):
        try:
            vImgData = io.BytesIO(requests.get(vImgUrl).content)
            vMovieSheet.insert_image('A5', "poster.jpg", {'image_data': vImgData})
            vHasImage = True
        except:
            vMovieSheet.write('A5', "Image download failed")
            
    # 2. Write Main Sheet Data
    vDateObj = pd.to_datetime(row['release_date']).to_pydatetime()
    
    vMainSheet.write_datetime(vRow, 0, vDateObj, fmt_date)
    vMainSheet.write(vRow, 1, row['movie_title'], fmt_text)
    
    # Hyperlink to the new sheet
    vLinkLocation = f"internal:'{vSheetName}'!A1"
    vMainSheet.write_url(vRow, 2, vLinkLocation, fmt_link, string="View Poster")
    
    vMainSheet.write(vRow, 3, row['genres'], fmt_text)
    vMainSheet.write(vRow, 4, row['cast_members'], fmt_text)
    
    vRow += 1

vMainSheet.autofilter(0, 0, vRow-1, 4)
vWorkbook.close()

print(f"Report generated: {vReportFilename}")
vCon.close()